In [2]:
import numpy as np
import pandas as pd
PREVIOUS_MAX_ROWS = pd.options.display.max_rows
pd.options.display.max_columns = 20
pd.options.display.max_rows = 20
pd.options.display.max_colwidth = 80
np.random.seed(12345)
import matplotlib.pyplot as plt
plt.rc("figure", figsize=(10, 6))
np.set_printoptions(precision=4, suppress=True)

In [3]:
import numpy as np
import pandas as pd

## 1. groupby의 기본 예제

In [4]:
df = pd.DataFrame({"key1" : ["a", "a", None, "b", "b", "a", None],
                    "key2" : pd.Series([1, 2, 1, 2, 1, None, 1],
                                        dtype="Int64"),
                    "data1" : np.random.standard_normal(7),
                    "data2" : np.random.standard_normal(7)})
df

,key1,key2,data1,data2
0,a,1,-0.204708,0.281746
1,a,2,0.478943,0.769023
2,None,1,-0.519439,1.246435
3,b,2,-0.555730,1.007189
4,b,1,1.965781,-1.296221
5,a,<NA>,1.393406,0.274992
6,None,1,0.092908,0.228913


In [5]:
df["data1"]

0   -0.204708
1    0.478943
2   -0.519439
3   -0.555730
4    1.965781
5    1.393406
6    0.092908
Name: data1, dtype: float64

In [6]:
df["key1"]

0       a
1       a
2    None
3       b
4       b
5       a
6    None
Name: key1, dtype: object

In [23]:
grouped = df["data1"].groupby(df["key1"])   # groupby에서 None값은 무시됨
grouped

In [8]:
grouped.mean()

key1
a    0.555881
b    0.705025
Name: data1, dtype: float64

In [9]:
means = df["data1"].groupby([df["key1"], df["key2"]]).mean()
means

key1  key2
a     1      -0.204708
      2       0.478943
b     1       1.965781
      2      -0.555730
Name: data1, dtype: float64

### 피봇테이블 예시

In [12]:
means.unstack()   # 가장 높은 레벨의 인덱스를 열로 옮김

key2,1,2
key1,,
a,-0.204708,0.478943
b,1.965781,-0.555730


### 다른 예시

In [17]:
states = np.array(["OH", "CA", "CA", "OH", "OH", "CA", "OH"])   # 넘파이 배열
years  = [2005, 2005, 2006, 2005, 2006, 2005, 2006]             # 리스트
df

,key1,key2,data1,data2
0,a,1,-0.204708,0.281746
1,a,2,0.478943,0.769023
2,None,1,-0.519439,1.246435
3,b,2,-0.555730,1.007189
4,b,1,1.965781,-1.296221
5,a,<NA>,1.393406,0.274992
6,None,1,0.092908,0.228913


In [18]:
df["data1"].groupby([states, years]).mean()

CA  2005    0.936175
    2006   -0.519439
OH  2005   -0.380219
    2006    1.029344
Name: data1, dtype: float64

#### 데이터 프레임에 대해 groupby 적용 (위는 series에 적용)

In [24]:
df.groupby("key1").mean()   # 키값으로 groupby 가능

,key2,data1,data2
key1,,,
a,1.5,0.555881,0.441920
b,1.5,0.705025,-0.144516


In [25]:
df.groupby("key2").mean(numeric_only=True)  # 숫자로 이루어진 피쳐만 나타냄

,data1,data2
key2,,
1,0.333636,0.115218
2,-0.038393,0.888106


In [26]:
df.groupby("key2").mean()       # 버전에 따라 오류일수도 아닐수도..

TypeError: agg function failed [how->mean,dtype->object]

In [27]:
df.groupby(["key1", "key2"]).mean()

data1     data2
key1 key2                    
a    1    -0.204708  0.281746
     2     0.478943  0.769023
b    1     1.965781 -1.296221
     2    -0.555730  1.007189

### groupby 객체의 size() 메소드

In [28]:
df

,key1,key2,data1,data2
0,a,1,-0.204708,0.281746
1,a,2,0.478943,0.769023
2,None,1,-0.519439,1.246435
3,b,2,-0.555730,1.007189
4,b,1,1.965781,-1.296221
5,a,<NA>,1.393406,0.274992
6,None,1,0.092908,0.228913


In [30]:
df.groupby(["key1", "key2"]).size()     # 몇 개 있는지 알려줌

key1  key2
a     1       1
      2       1
b     1       1
      2       1
dtype: int64

In [33]:
df.groupby("key1", dropna=False).size()     # 결측치 포함

key1
a      3
b      2
NaN    2
dtype: int64

In [34]:
df.groupby(["key1", "key2"], dropna=False).size()   # 결측치 포함

key1  key2
a     1       1
      2       1
      <NA>    1
b     1       1
      2       1
NaN   1       2
dtype: int64

### groupby객체의 count() 메소드

In [36]:
df

,key1,key2,data1,data2
0,a,1,-0.204708,0.281746
1,a,2,0.478943,0.769023
2,None,1,-0.519439,1.246435
3,b,2,-0.555730,1.007189
4,b,1,1.965781,-1.296221
5,a,<NA>,1.393406,0.274992
6,None,1,0.092908,0.228913


In [38]:
df.groupby("key1").count()  # 결측값 제외하고 'key1'이 각각의 column에 몇개 있는지

,key2,data1,data2
key1,,,
a,2,3,3
b,2,2,2


## 2. 그룹간 순회하기

### 그룹 이름과 해당 그룹으로 구성된 데이터프레임을 반환

In [68]:
for name, group in df.groupby("key1"):
    print(name)
    print(group)


a
  key1  key2     data1     data2
0    a     1 -1.541996  1.349742
1    a     2 -0.970736  0.069877
5    a  <NA> -0.753887  1.327195
b
  key1  key2     data1     data2
3    b     2  0.286350 -0.011862
4    b     1  0.377984  1.004812


In [69]:
for (k1, k2), group in df.groupby(["key1", "key2"]):
    print((k1, k2))
    print(group)


('a', 1)
  key1  key2     data1     data2
0    a     1 -1.541996  1.349742
('a', 2)
  key1  key2     data1     data2
1    a     2 -0.970736  0.069877
('b', 1)
  key1  key2     data1     data2
4    b     1  0.377984  1.004812
('b', 2)
  key1  key2    data1     data2
3    b     2  0.28635 -0.011862


### 딕셔너리를 만들어서 활용 가능

In [71]:
pieces = {name: group for name, group in df.groupby("key1")}
pieces["b"]

,key1,key2,data1,data2
3,b,2,0.286350,-0.011862
4,b,1,0.377984,1.004812


## 3. 딕셔너리를 통해 그룹화 하는 방법

In [72]:
df

,key1,key2,data1,data2
0,a,1,-1.541996,1.349742
1,a,2,-0.970736,0.069877
2,None,1,-1.307030,0.246674
3,b,2,0.286350,-0.011862
4,b,1,0.377984,1.004812
5,a,<NA>,-0.753887,1.327195
6,None,1,0.331286,-0.919262


In [74]:
grouped = df.groupby({0:"low", 1:"low",
                      2:"low", 3:"low",4:"high", 5:"high",
                      6:"high"})
for group_key, group_values in grouped:
    print(group_key)
    print(group_values)

high
   key1  key2     data1     data2
4     b     1  0.377984  1.004812
5     a  <NA> -0.753887  1.327195
6  None     1  0.331286 -0.919262
low
   key1  key2     data1     data2
0     a     1 -1.541996  1.349742
1     a     2 -0.970736  0.069877
2  None     1 -1.307030  0.246674
3     b     2  0.286350 -0.011862


### 열을 그룹화 하는 방법 

In [77]:
grouped = df.groupby({"key1": "key", "key2": "key",
                      "data1": "data", "data2": "data"}, axis="columns")

In [78]:
for group_key, group_values in grouped:
    print(group_key)
    print(group_values)


data
      data1     data2
0 -1.541996  1.349742
1 -0.970736  0.069877
2 -1.307030  0.246674
3  0.286350 -0.011862
4  0.377984  1.004812
5 -0.753887  1.327195
6  0.331286 -0.919262
key
   key1  key2
0     a     1
1     a     2
2  None     1
3     b     2
4     b     1
5     a  <NA>
6  None     1


## 4. 열의 일부 선택하기

In [80]:
df.groupby(["key1", "key2"])[["data2"]].mean()  #data1은 선택되지 않음

data2
key1 key2          
a    1     1.349742
     2     0.069877
b    1     1.004812
     2    -0.011862

In [81]:
s_grouped = df.groupby(["key1", "key2"])["data2"]
s_grouped

In [82]:
s_grouped.mean()

key1  key2
a     1       1.349742
      2       0.069877
b     1       1.004812
      2      -0.011862
Name: data2, dtype: float64

## 5. 다양한 방식을 그룹화하기

In [84]:
people = pd.DataFrame(np.random.standard_normal((5, 5)),
                      columns=["a", "b", "c", "d", "e"],
                      index=["Joe", "Steve", "Wanda", "Jill", "Trey"])
people.iloc[2:3, [1, 2]] = np.nan # Add a few NA values
people

,a,b,c,d,e
Joe,-1.157719,0.816707,0.433610,1.010737,1.824875
Steve,-0.997518,0.850591,-0.131578,0.912414,0.188211
Wanda,2.169461,NaN,NaN,0.029610,0.795253
Jill,0.118110,-0.748532,0.584970,0.152677,-1.565657
Trey,-0.562540,-0.032664,-0.929006,-0.482573,-0.036264


### 딕셔너리를 통한 그룹화

In [86]:
mapping = {"a": "red", "b": "red", "c": "blue",
           "d": "blue", "e": "red", "f" : "orange"}

In [87]:
by_column = people.groupby(mapping, axis="columns")
by_column.sum()

,blue,red
Joe,1.444347,1.483863
Steve,0.780837,0.041284
Wanda,0.029610,2.964715
Jill,0.737646,-2.196079
Trey,-1.411579,-0.631468


### 시리즈를 통한 그룹화

In [88]:
map_series = pd.Series(mapping)
map_series
people.groupby(map_series, axis="columns").count()

,blue,red
Joe,2,3
Steve,2,3
Wanda,1,2
Jill,2,3
Trey,2,3


### 함수를 통한 그룹화

In [89]:
people.groupby(len).sum()

,a,b,c,d,e
3,-1.157719,0.816707,0.433610,1.010737,1.824875
4,-0.444430,-0.781196,-0.344036,-0.329896,-1.601921
5,1.171943,0.850591,-0.131578,0.942024,0.983464


### 함수와 시리즈를 섞어서 그룹화

In [90]:
key_list = ["one", "one", "one", "two", "two"]
people.groupby([len, key_list]).min()

,,a,b,c,d,e
3,one,-1.157719,0.816707,0.433610,1.010737,1.824875
4,two,-0.562540,-0.748532,-0.929006,-0.482573,-1.565657
5,one,-0.997518,0.850591,-0.131578,0.029610,0.188211


## 6. 데이터 집계

In [91]:
df

,key1,key2,data1,data2
0,a,1,-1.541996,1.349742
1,a,2,-0.970736,0.069877
2,None,1,-1.307030,0.246674
3,b,2,0.286350,-0.011862
4,b,1,0.377984,1.004812
5,a,<NA>,-0.753887,1.327195
6,None,1,0.331286,-0.919262


In [92]:
grouped = df.groupby("key1")
grouped["data1"].nsmallest(2)

key1   
a     0   -1.541996
      1   -0.970736
b     3    0.286350
      4    0.377984
Name: data1, dtype: float64

In [94]:
def peak_to_peak(arr):
    return arr.max() - arr.min()
grouped.agg(peak_to_peak)

,key2,data1,data2
key1,,,
a,1,0.788109,1.279866
b,1,0.091634,1.016673


## 7. 데이터 집계 관련 다양한 예시

In [95]:
tips = pd.read_csv("examples/tips.csv")
tips.head()

,total_bill,tip,smoker,day,time,size
0,16.99,1.01,No,Sun,Dinner,2
1,10.34,1.66,No,Sun,Dinner,3
2,21.01,3.50,No,Sun,Dinner,3
3,23.68,3.31,No,Sun,Dinner,2
4,24.59,3.61,No,Sun,Dinner,4


In [96]:
tips["tip_pct"] = tips["tip"] / tips["total_bill"]
tips.head()

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808


In [97]:
grouped = tips.groupby(["day", "smoker"])

### agg(메소드 이름 혹은 함수) 를 통해서도 집계 가능

In [98]:
grouped_pct = grouped["tip_pct"]
grouped_pct.agg("mean")

day   smoker
Fri   No        0.151650
      Yes       0.174783
Sat   No        0.158048
      Yes       0.147906
Sun   No        0.160113
      Yes       0.187250
Thur  No        0.160298
      Yes       0.163863
Name: tip_pct, dtype: float64

### 메소드 이름과 함수 배열을 통해 여러 집계 값을 나타낼 수 있음

In [99]:
grouped_pct.agg(["mean", "std", peak_to_peak])

mean       std  peak_to_peak
day  smoker                                  
Fri  No      0.151650  0.028123      0.067349
     Yes     0.174783  0.051293      0.159925
Sat  No      0.158048  0.039767      0.235193
     Yes     0.147906  0.061375      0.290095
Sun  No      0.160113  0.042347      0.193226
     Yes     0.187250  0.154134      0.644685
Thur No      0.160298  0.038774      0.193350
     Yes     0.163863  0.039389      0.151240

### 튜플 형태로 적용되는 함수의 이름을 나타낼 수 있음

In [100]:
grouped_pct.agg([("average", "mean"), ("stdev", np.std)])

average     stdev
day  smoker                    
Fri  No      0.151650  0.028123
     Yes     0.174783  0.051293
Sat  No      0.158048  0.039767
     Yes     0.147906  0.061375
Sun  No      0.160113  0.042347
     Yes     0.187250  0.154134
Thur No      0.160298  0.038774
     Yes     0.163863  0.039389

### 여러 개의 열에 적용 가능 하고,  딕셔너리를 통해서 열마다 서로 다른 함수도 적용 가능

In [102]:
functions = ["count", "mean", "max"]
result = grouped[["tip_pct", "total_bill"]].agg(functions)
result

tip_pct                     total_bill                  
              count      mean       max      count       mean    max
day  smoker                                                         
Fri  No           4  0.151650  0.187735          4  18.420000  22.75
     Yes         15  0.174783  0.263480         15  16.813333  40.17
Sat  No          45  0.158048  0.291990         45  19.661778  48.33
     Yes         42  0.147906  0.325733         42  21.276667  50.81
Sun  No          57  0.160113  0.252672         57  20.506667  48.17
     Yes         19  0.187250  0.710345         19  24.120000  45.35
Thur No          45  0.160298  0.266312         45  17.113111  41.19
     Yes         17  0.163863  0.241255         17  19.190588  43.11

In [104]:
ftuples = [("Average", "mean"), ("Variance", np.var)]
grouped[["tip_pct", "total_bill"]].agg(ftuples)

tip_pct           total_bill            
              Average  Variance    Average    Variance
day  smoker                                           
Fri  No      0.151650  0.000791  18.420000   25.596333
     Yes     0.174783  0.002631  16.813333   82.562438
Sat  No      0.158048  0.001581  19.661778   79.908965
     Yes     0.147906  0.003767  21.276667  101.387535
Sun  No      0.160113  0.001793  20.506667   66.099980
     Yes     0.187250  0.023757  24.120000  109.046044
Thur No      0.160298  0.001503  17.113111   59.625081
     Yes     0.163863  0.001551  19.190588   69.808518

In [105]:
grouped.agg({"tip" : np.max, "size" : "sum"})

tip  size
day  smoker             
Fri  No       3.50     9
     Yes      4.73    31
Sat  No       9.00   115
     Yes     10.00   104
Sun  No       6.00   167
     Yes      6.50    49
Thur No       6.70   112
     Yes      5.00    40

In [106]:
grouped.agg({"tip_pct" : ["min", "max", "mean", "std"],
             "size" : "sum"})

tip_pct                               size
                  min       max      mean       std  sum
day  smoker                                             
Fri  No      0.120385  0.187735  0.151650  0.028123    9
     Yes     0.103555  0.263480  0.174783  0.051293   31
Sat  No      0.056797  0.291990  0.158048  0.039767  115
     Yes     0.035638  0.325733  0.147906  0.061375  104
Sun  No      0.059447  0.252672  0.160113  0.042347  167
     Yes     0.065660  0.710345  0.187250  0.154134   49
Thur No      0.072961  0.266312  0.160298  0.038774  112
     Yes     0.090014  0.241255  0.163863  0.039389   40

In [108]:
grouped = tips.groupby(["day", "smoker"])
grouped.mean(numeric_only=True)

total_bill       tip      size   tip_pct
day  smoker                                          
Fri  No       18.420000  2.812500  2.250000  0.151650
     Yes      16.813333  2.714000  2.066667  0.174783
Sat  No       19.661778  3.102889  2.555556  0.158048
     Yes      21.276667  2.875476  2.476190  0.147906
Sun  No       20.506667  3.167895  2.929825  0.160113
     Yes      24.120000  3.516842  2.578947  0.187250
Thur No       17.113111  2.673778  2.488889  0.160298
     Yes      19.190588  3.030000  2.352941  0.163863

In [116]:
grouped = tips.groupby(["day", "smoker"], as_index=False)  ## as_index 옵션을 통해서 인덱스 작업을 하지 않을 수 있음
grouped.mean(numeric_only=True)

,day,smoker,total_bill,tip,size,tip_pct
0,Fri,No,18.420000,2.812500,2.250000,0.151650
1,Fri,Yes,16.813333,2.714000,2.066667,0.174783
2,Sat,No,19.661778,3.102889,2.555556,0.158048
3,Sat,Yes,21.276667,2.875476,2.476190,0.147906
4,Sun,No,20.506667,3.167895,2.929825,0.160113
5,Sun,Yes,24.120000,3.516842,2.578947,0.187250
6,Thur,No,17.113111,2.673778,2.488889,0.160298
7,Thur,Yes,19.190588,3.030000,2.352941,0.163863


## 8. apply 메서드

In [117]:
def top(df, n=5, column="tip_pct"):
    return df.sort_values(column, ascending=False)[:n]
top(tips, n=6)

,total_bill,tip,smoker,day,time,size,tip_pct
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
232,11.61,3.39,No,Sat,Dinner,2,0.291990
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525


In [118]:
tips.groupby("smoker").apply(top)

total_bill   tip smoker   day    time  size   tip_pct
smoker                                                           
No     232       11.61  3.39     No   Sat  Dinner     2  0.291990
       149        7.51  2.00     No  Thur   Lunch     2  0.266312
       51        10.29  2.60     No   Sun  Dinner     2  0.252672
       185       20.69  5.00     No   Sun  Dinner     5  0.241663
       88        24.71  5.85     No  Thur   Lunch     2  0.236746
Yes    172        7.25  5.15    Yes   Sun  Dinner     2  0.710345
       178        9.60  4.00    Yes   Sun  Dinner     2  0.416667
       67         3.07  1.00    Yes   Sat  Dinner     1  0.325733
       183       23.17  6.50    Yes   Sun  Dinner     4  0.280535
       109       14.31  4.00    Yes   Sat  Dinner     2  0.279525

In [119]:
tips.groupby(["smoker", "day"]).apply(top, n=1, column="total_bill") 

total_bill    tip smoker   day    time  size   tip_pct
smoker day                                                             
No     Fri  94        22.75   3.25     No   Fri  Dinner     2  0.142857
       Sat  212       48.33   9.00     No   Sat  Dinner     4  0.186220
       Sun  156       48.17   5.00     No   Sun  Dinner     6  0.103799
       Thur 142       41.19   5.00     No  Thur   Lunch     5  0.121389
Yes    Fri  95        40.17   4.73    Yes   Fri  Dinner     4  0.117750
       Sat  170       50.81  10.00    Yes   Sat  Dinner     3  0.196812
       Sun  182       45.35   3.50    Yes   Sun  Dinner     3  0.077178
       Thur 197       43.11   5.00    Yes  Thur   Lunch     4  0.115982

### group_keys=False를 통해서 인덱싱 작업을 하지 않을 수 있음

In [121]:
tips.groupby(["smoker", "day"], group_keys=False).apply(top, n=1, column="total_bill")  

,total_bill,tip,smoker,day,time,size,tip_pct
94,22.75,3.25,No,Fri,Dinner,2,0.142857
212,48.33,9.00,No,Sat,Dinner,4,0.186220
156,48.17,5.00,No,Sun,Dinner,6,0.103799
142,41.19,5.00,No,Thur,Lunch,5,0.121389
95,40.17,4.73,Yes,Fri,Dinner,4,0.117750
170,50.81,10.00,Yes,Sat,Dinner,3,0.196812
182,45.35,3.50,Yes,Sun,Dinner,3,0.077178
197,43.11,5.00,Yes,Thur,Lunch,4,0.115982


## 9. groupby를 활용한 예시 1 - 사분위수 분석, 버킷 분석

In [122]:
frame = pd.DataFrame({"data1": np.random.standard_normal(1000),
                      "data2": np.random.standard_normal(1000)})
frame.head()

,data1,data2
0,1.095390,-0.154208
1,0.980928,0.921908
2,-0.589488,1.956463
3,1.581700,0.379706
4,-0.528735,0.270780


In [123]:
quartiles = pd.cut(frame["data1"], 4)
quartiles.head(10)

0     (0.489, 2.208]
1     (0.489, 2.208]
2     (-1.23, 0.489]
3     (0.489, 2.208]
4     (-1.23, 0.489]
5     (-1.23, 0.489]
6     (0.489, 2.208]
7    (-2.956, -1.23]
8     (-1.23, 0.489]
9     (-1.23, 0.489]
Name: data1, dtype: category
Categories (4, interval[float64]): [(-2.956, -1.23] < (-1.23, 0.489] < (0.489, 2.208] < (2.208, 3.928]]

In [124]:
def get_stats(group):
    return pd.DataFrame(
        {"min": group.min(), "max": group.max(),
        "count": group.count(), "mean": group.mean()}
    )

grouped = frame.groupby(quartiles)  ## category를 groupby의 기준으로 삼을 수 있음
grouped.apply(get_stats)

min       max  count      mean
data1                                                     
(-2.956, -1.23] data1 -2.949343 -1.230179     95 -1.662006
                data2 -3.399312  1.670835     95 -0.041821
(-1.23, 0.489]  data1 -1.228918  0.488675    601 -0.327033
                data2 -2.989741  3.260383    601 -0.015011
(0.489, 2.208]  data1  0.489965  2.200997    293  1.069111
                data2 -3.745356  2.954439    293  0.089964
(2.208, 3.928]  data1  2.212303  3.927528     11  2.622638
                data2 -1.929776  1.765640     11  0.055158

In [125]:
grouped.agg(["min", "max", "count", "mean"])

data1                               data2                  \
                      min       max count      mean       min       max count   
data1                                                                           
(-2.956, -1.23] -2.949343 -1.230179    95 -1.662006 -3.399312  1.670835    95   
(-1.23, 0.489]  -1.228918  0.488675   601 -0.327033 -2.989741  3.260383   601   
(0.489, 2.208]   0.489965  2.200997   293  1.069111 -3.745356  2.954439   293   
(2.208, 3.928]   2.212303  3.927528    11  2.622638 -1.929776  1.765640    11   

                           
                     mean  
data1                      
(-2.956, -1.23] -0.041821  
(-1.23, 0.489]  -0.015011  
(0.489, 2.208]   0.089964  
(2.208, 3.928]   0.055158

In [126]:
quartiles_samp = pd.qcut(frame["data1"], 4, labels=False)
quartiles_samp.head()

0    3
1    3
2    1
3    3
4    1
Name: data1, dtype: int64

In [127]:
grouped = frame.groupby(quartiles_samp)
grouped.apply(get_stats)

min       max  count      mean
data1                                           
0     data1 -2.949343 -0.685484    250 -1.213667
      data2 -3.399312  2.628441    250 -0.045859
1     data1 -0.683066 -0.050667    250 -0.374976
      data2 -2.630247  3.260383    250 -0.052155
2     data1 -0.048565  0.609512    250  0.288155
      data2 -2.989741  2.458842    250  0.044172
3     data1  0.611953  3.927528    250  1.251132
      data2 -3.745356  2.954439    250  0.109728

## 10. groupby를 활용한 예시 2 - 그룹별 값으로 결측치 채우기

In [129]:
s = pd.Series(np.random.standard_normal(6))
s[::2] = np.nan
s

0         NaN
1   -0.057191
2         NaN
3    0.310821
4         NaN
5    1.308413
dtype: float64

In [130]:
s.fillna(s.mean())

0    0.520681
1   -0.057191
2    0.520681
3    0.310821
4    0.520681
5    1.308413
dtype: float64

In [131]:
states = ["Ohio", "New York", "Vermont", "Florida",
          "Oregon", "Nevada", "California", "Idaho"]
group_key = ["East", "East", "East", "East",
             "West", "West", "West", "West"]
data = pd.Series(np.random.standard_normal(8), index=states)
data

Ohio         -0.355505
New York      0.807641
Vermont       0.104932
Florida       0.504662
Oregon       -1.182779
Nevada        0.057679
California    0.316274
Idaho        -1.766809
dtype: float64

In [132]:
data[["Vermont", "Nevada", "Idaho"]] = np.nan
data

Ohio         -0.355505
New York      0.807641
Vermont            NaN
Florida       0.504662
Oregon       -1.182779
Nevada             NaN
California    0.316274
Idaho              NaN
dtype: float64

In [134]:
data.groupby(group_key).size()

East    4
West    4
dtype: int64

In [135]:
data.groupby(group_key).count()

East    3
West    2
dtype: int64

In [136]:
data.groupby(group_key).mean()

East    0.318933
West   -0.433253
dtype: float64

In [137]:
def fill_mean(group):
    return group.fillna(group.mean())

data.groupby(group_key).apply(fill_mean)

Ohio         -0.355505
New York      0.807641
Vermont       0.318933
Florida       0.504662
Oregon       -1.182779
Nevada       -0.433253
California    0.316274
Idaho        -0.433253
dtype: float64

In [138]:
fill_values = {"East": 0.5, "West": -1}
def fill_func(group):
    return group.fillna(fill_values[group.name])                ## 각 group의 name attribute가 있음

data.groupby(group_key).apply(fill_func)

Ohio         -0.355505
New York      0.807641
Vermont       0.500000
Florida       0.504662
Oregon       -1.182779
Nevada       -1.000000
California    0.316274
Idaho        -1.000000
dtype: float64

## 11. groupby를 활용한 예시 3 - 랜덤 표본과 순열

In [139]:
suits = ["H", "S", "C", "D"]  # Hearts, Spades, Clubs, Diamonds
card_val = (list(range(1, 11)) + [10] * 3) * 4
base_names = ["A"] + list(range(2, 11)) + ["J", "K", "Q"]
cards = []
for suit in suits:
    cards.extend(str(num) + suit for num in base_names)

deck = pd.Series(card_val, index=cards)

In [141]:
deck.head(15)

AH      1
2H      2
3H      3
4H      4
5H      5
6H      6
7H      7
8H      8
9H      9
10H    10
JH     10
KH     10
QH     10
AS      1
2S      2
dtype: int64

In [142]:
def draw(deck, n=5):
    return deck.sample(n)
draw(deck)

KD     10
5D      5
3H      3
10S    10
9H      9
dtype: int64

In [143]:
def get_suit(card):
    # last letter is suit
    return card[-1]

deck.groupby(get_suit).apply(draw, n=2)

C  10C    10
   8C      8
D  QD     10
   2D      2
H  3H      3
   4H      4
S  7S      7
   8S      8
dtype: int64

In [144]:
deck.groupby(get_suit, group_keys=False).apply(draw, n=2)

9C      9
3C      3
5D      5
2D      2
8H      8
JH     10
10S    10
3S      3
dtype: int64

## 12. transform 

In [157]:
df = pd.DataFrame({'key': ['a', 'b', 'c'] * 4,
                   'value': np.arange(12.)})
df

,key,value
0,a,0.0
1,b,1.0
2,c,2.0
3,a,3.0
4,b,4.0
5,c,5.0
6,a,6.0
7,b,7.0
8,c,8.0
9,a,9.0


In [158]:
g = df.groupby('key')['value']
g.mean()

key
a    4.5
b    5.5
c    6.5
Name: value, dtype: float64

In [159]:
def get_mean(group):
    return group.mean()
g.transform(get_mean)

0     4.5
1     5.5
2     6.5
3     4.5
4     5.5
5     6.5
6     4.5
7     5.5
8     6.5
9     4.5
10    5.5
11    6.5
Name: value, dtype: float64

In [160]:
g.transform('mean')

0     4.5
1     5.5
2     6.5
3     4.5
4     5.5
5     6.5
6     4.5
7     5.5
8     6.5
9     4.5
10    5.5
11    6.5
Name: value, dtype: float64

In [161]:
g.apply('mean')

TypeError: 'str' object is not callable

In [162]:
g.apply(get_mean)

key
a    4.5
b    5.5
c    6.5
Name: value, dtype: float64

In [163]:
def times_two(group):
    return group * 2
g.transform(times_two)

0      0.0
1      2.0
2      4.0
3      6.0
4      8.0
5     10.0
6     12.0
7     14.0
8     16.0
9     18.0
10    20.0
11    22.0
Name: value, dtype: float64

In [164]:
def get_ranks(group):
    return group.rank(ascending=False)
g.transform(get_ranks)

0     4.0
1     4.0
2     4.0
3     3.0
4     3.0
5     3.0
6     2.0
7     2.0
8     2.0
9     1.0
10    1.0
11    1.0
Name: value, dtype: float64

In [165]:
def normalize(x):
    return (x - x.mean()) / x.std()

In [166]:
g.transform(normalize)

0    -1.161895
1    -1.161895
2    -1.161895
3    -0.387298
4    -0.387298
5    -0.387298
6     0.387298
7     0.387298
8     0.387298
9     1.161895
10    1.161895
11    1.161895
Name: value, dtype: float64

In [168]:
g.apply(normalize)  ## 같은 결과를 얻음

0    -1.161895
1    -1.161895
2    -1.161895
3    -0.387298
4    -0.387298
5    -0.387298
6     0.387298
7     0.387298
8     0.387298
9     1.161895
10    1.161895
11    1.161895
Name: value, dtype: float64

## 13. 피벗 테이블

In [196]:
tips.head()

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808


In [171]:
tips.pivot_table(index=["day", "smoker"],
                 values=["size", "tip", "tip_pct", "total_bill"]) ## 기본은 평균치를 집계함 

size       tip   tip_pct  total_bill
day  smoker                                          
Fri  No      2.250000  2.812500  0.151650   18.420000
     Yes     2.066667  2.714000  0.174783   16.813333
Sat  No      2.555556  3.102889  0.158048   19.661778
     Yes     2.476190  2.875476  0.147906   21.276667
Sun  No      2.929825  3.167895  0.160113   20.506667
     Yes     2.578947  3.516842  0.187250   24.120000
Thur No      2.488889  2.673778  0.160298   17.113111
     Yes     2.352941  3.030000  0.163863   19.190588

In [175]:
tips.pivot_table(index=["time", "day"], columns=["smoker","size"],
                 values=["tip_pct"])

tip_pct                                                    \
smoker             No                                                     
size                1         2         3         4         5         6   
time   day                                                                
Dinner Fri        NaN  0.139622       NaN       NaN       NaN       NaN   
       Sat   0.137931  0.162705  0.154661  0.150096       NaN       NaN   
       Sun        NaN  0.168859  0.152663  0.148143  0.206928  0.103799   
       Thur       NaN  0.159744       NaN       NaN       NaN       NaN   
Lunch  Fri        NaN       NaN  0.187735       NaN       NaN       NaN   
       Thur  0.181728  0.166005  0.084246  0.138919  0.121389  0.173706   

                                                               
smoker            Yes                                          
size                1         2         3         4         5  
time   day                                                     
Dinner Fri        NaN  0.171297       NaN  0.117750       NaN  
       Sat   0.325733  0.148668  0.144995  0.124515  0.106572  
       Sun        NaN  0.207893  0.152660  0.193370  0.065660  
       Thur       NaN       NaN       NaN       NaN       NaN  
Lunch  Fri   0.223776  0.181969       NaN       NaN       NaN  
       Thur       NaN  0.158843  0.204952  0.155410       NaN

In [176]:
tips.pivot_table(index=["time", "day"], columns="smoker",
                 values=["tip_pct","size"])

size             tip_pct          
smoker             No       Yes        No       Yes
time   day                                         
Dinner Fri   2.000000  2.222222  0.139622  0.165347
       Sat   2.555556  2.476190  0.158048  0.147906
       Sun   2.929825  2.578947  0.160113  0.187250
       Thur  2.000000       NaN  0.159744       NaN
Lunch  Fri   3.000000  1.833333  0.187735  0.188937
       Thur  2.500000  2.352941  0.160311  0.163863

In [177]:
tips.pivot_table(index=["time", "day"], columns="smoker",
                 values=["tip_pct", "size"], margins=True)

size                       tip_pct                    
smoker             No       Yes       All        No       Yes       All
time   day                                                             
Dinner Fri   2.000000  2.222222  2.166667  0.139622  0.165347  0.158916
       Sat   2.555556  2.476190  2.517241  0.158048  0.147906  0.153152
       Sun   2.929825  2.578947  2.842105  0.160113  0.187250  0.166897
       Thur  2.000000       NaN  2.000000  0.159744       NaN  0.159744
Lunch  Fri   3.000000  1.833333  2.000000  0.187735  0.188937  0.188765
       Thur  2.500000  2.352941  2.459016  0.160311  0.163863  0.161301
All          2.668874  2.408602  2.569672  0.159328  0.163196  0.160803

In [186]:
tips.pivot_table(index=["time", "smoker"], columns="day",
                 values="tip_pct", aggfunc=len, margins=True)

day             Fri   Sat   Sun  Thur    All
time   smoker                               
Dinner No       3.0  45.0  57.0   1.0  106.0
       Yes      9.0  42.0  19.0   NaN   70.0
Lunch  No       1.0   NaN   NaN  44.0   45.0
       Yes      6.0   NaN   NaN  17.0   23.0
All            19.0  87.0  76.0  62.0  244.0

In [187]:
tips.pivot_table(index=["time", "smoker"], columns="day",
                 values="tip_pct", aggfunc="count", margins=True)

day             Fri   Sat   Sun  Thur  All
time   smoker                             
Dinner No       3.0  45.0  57.0   1.0  106
       Yes      9.0  42.0  19.0   NaN   70
Lunch  No       1.0   NaN   NaN  44.0   45
       Yes      6.0   NaN   NaN  17.0   23
All            19.0  87.0  76.0  62.0  244

In [188]:
tips.pivot_table(index=["time", "size", "smoker"], columns="day",
                 values="tip_pct", fill_value=0)

day                      Fri       Sat       Sun      Thur
time   size smoker                                        
Dinner 1    No      0.000000  0.137931  0.000000  0.000000
            Yes     0.000000  0.325733  0.000000  0.000000
       2    No      0.139622  0.162705  0.168859  0.159744
            Yes     0.171297  0.148668  0.207893  0.000000
       3    No      0.000000  0.154661  0.152663  0.000000
...                      ...       ...       ...       ...
Lunch  3    Yes     0.000000  0.000000  0.000000  0.204952
       4    No      0.000000  0.000000  0.000000  0.138919
            Yes     0.000000  0.000000  0.000000  0.155410
       5    No      0.000000  0.000000  0.000000  0.121389
       6    No      0.000000  0.000000  0.000000  0.173706

[21 rows x 4 columns]

## 14. 교차표 (crosstab)

In [189]:
from io import StringIO
data = """Sample  Nationality  Handedness
1   USA  Right-handed
2   Japan    Left-handed
3   USA  Right-handed
4   Japan    Right-handed
5   Japan    Left-handed
6   Japan    Right-handed
7   USA  Right-handed
8   USA  Left-handed
9   Japan    Right-handed
10  USA  Right-handed"""
data = pd.read_table(StringIO(data), sep="\s+")

In [190]:
data

,Sample,Nationality,Handedness
0,1,USA,Right-handed
1,2,Japan,Left-handed
2,3,USA,Right-handed
3,4,Japan,Right-handed
4,5,Japan,Left-handed
5,6,Japan,Right-handed
6,7,USA,Right-handed
7,8,USA,Left-handed
8,9,Japan,Right-handed
9,10,USA,Right-handed


In [191]:
pd.crosstab(data["Nationality"], data["Handedness"], margins=True)

Handedness,Left-handed,Right-handed,All
Nationality,,,
Japan,2,3,5
USA,1,4,5
All,3,7,10


In [215]:
data.pivot_table(index='Nationality', columns='Handedness',values='Sample',aggfunc='count',margins=True)

Handedness,Left-handed,Right-handed,All
Nationality,,,
Japan,2,3,5
USA,1,4,5
All,3,7,10


In [192]:
pd.crosstab([tips["time"], tips["day"]], tips["smoker"], margins=True)

smoker        No  Yes  All
time   day                
Dinner Fri     3    9   12
       Sat    45   42   87
       Sun    57   19   76
       Thur    1    0    1
Lunch  Fri     1    6    7
       Thur   44   17   61
All          151   93  244

In [213]:
tips.pivot_table(index=['time','day'], columns='smoker', values='total_bill',aggfunc='count',margins=True)

smoker          No   Yes  All
time   day                   
Dinner Fri     3.0   9.0   12
       Sat    45.0  42.0   87
       Sun    57.0  19.0   76
       Thur    1.0   NaN    1
Lunch  Fri     1.0   6.0    7
       Thur   44.0  17.0   61
All          151.0  93.0  244